# Data import

To start the database used in this notebook, please run `docker run -d --name pgData -p 5432:5432 postgres:latest`

To stop the database (and remove it), run `docker stop pgData && docker rm pgData`

In [ ]:
import psycopg2
from psycopg2.extensions import ISOLATION_LEVEL_AUTOCOMMIT
import csv

# Connect to an existing database
conn = psycopg2.connect("host=postgresdb user=postgres password=postgres")
conn.set_isolation_level(ISOLATION_LEVEL_AUTOCOMMIT)
# Open a cursor to perform database operations
cur = conn.cursor()
# Execute a command: this creates a new table
try:
    cur.execute("CREATE DATABASE mydata;")
except:
    print("Error while creating database, does it already exist?")
# Close database connection
conn.close()

In [ ]:
# Connect to an existing database
conn = psycopg2.connect("host=postgresdb user=postgres dbname=mydata password=postgres")
conn.set_isolation_level(ISOLATION_LEVEL_AUTOCOMMIT)
# Open a cursor to perform database operations
cur = conn.cursor()
# Execute a command: this creates a new table
try:
    cur.execute("""CREATE TABLE thunder (
    ID text PRIMARY KEY,
    PatientID text,
    DoB text,
    Date_Intake text,
    Clinical_T_Stage text,
    Clinical_N_Stage text,
    Age text,
    First_Fraction text,
    Last_Fraction text,
    TotalDose text,
    surgery text,
    surgery_type text,
    surgery_date text,
    Pathologic_T_stage text,
    Pathologic_N_stage text,
    TRG text,
    who text,
    Sex text);
    """)
except:
    print("Error while creating table lung1, does it already exist?")
# Close database connection
conn.close()

In [ ]:
import pandas
# Connect to an existing database
conn = psycopg2.connect("host=postgresdb user=postgres dbname=mydata password=postgres")
cur = conn.cursor()

#open CSV file
df = pandas.read_csv('https://raw.githubusercontent.com/jvsoest/Data-Integration-Tutorial/master/data/original/thunderSubSet.csv')
for index, row in df.iterrows():
    #print(str(index) + " | " + row.Idfu)
    #cur.execute
    cur.execute("""INSERT INTO thunder (ID, PatientID, Age, Clinical_T_Stage, Clinical_N_Stage, who, Sex, DoB, Date_Intake, First_Fraction, Last_Fraction, TotalDose, surgery, surgery_type, surgery_date, Pathologic_T_stage, Pathologic_N_stage, TRG) 
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)""",
          (index, row.Idfu, row.Age, row.cT, row.cN, row.WHO, row.Gender, row["Date.of.birth"], row["Date.Intake"], row.FF, row.LF, row.TotalDose, row.OK, row["Ok.type"], row["Date.OK"], row["pT.stage"], row["pN.stage"], row["TRG..1.4."]))
conn.commit()

In [ ]:
# Connect to an existing database
conn = psycopg2.connect("host=postgresdb user=postgres dbname=mydata password=postgres")
cur = conn.cursor()

cur.execute("""SELECT *
    FROM thunder;""")

results = cur.fetchall()
conn.close()

for row in results:
    print(row)